# Integrantes e Link do Vídeo

* Bernardo Maia Coelho - 12542481
* Gustavo Wadas Lopes - 12745640
* Pedro Guilherme dos Reis Teixeira - 12542477
* Pedro Henrique Vilela do Nascimento - 12803492

[Link do vídeo do trabalho 1](https://youtu.be/QR3MZX6I77E)

# Setup

### Libraries

In [ ]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from typing import TypedDict
from PIL import Image

### Abre a Janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Vertex e Fragment Shaders

In [ ]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = normals;            
        }
        """

fragment_code = """
        uniform vec4 color;
        uniform sampler2D samplerTexture;
        
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        vec3 lightColor = vec3(1.0, 1.0, 1.0);

        varying vec3 out_fragPos; // recebido do vertex shader
        varying vec2 out_texture;
        varying vec3 out_normal; // recebido do vertex shader
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

In [ ]:
# Request a program and shader slots from GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [ ]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)

# Matriz Model, View, Projection

In [ ]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)

    matrix_transform = glm.mat4(1.0)  # instanciando uma matriz identidade

    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))

    # aplicando rotacao
    matrix_transform = glm.rotate(
        matrix_transform, angle, glm.vec3(r_x, r_y, r_z))

    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))

    # pegando a transposta da matriz (glm trabalha com ela invertida)
    matrix_transform = np.array(matrix_transform)

    return matrix_transform


def view():
    global camera_pos, camera_front, camera_up
    mat_view = glm.lookAt(camera_pos, camera_pos + camera_front, camera_up)

    mat_view = np.array(mat_view)
    return mat_view


def projection():
    global altura, largura, inc_fov, inc_near, inc_far
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(
        glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)
    return mat_projection

# Modelos e Texturas

In [ ]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """

    vertices = []
    texture_coords = []
    normals = []
    faces = []

    material = None

    for line in open(filename, "r"):
        line = line.split("#")[0] # remove os comentários
        values = line.strip().split() # quebra a linha por espaço
        
        if len(values) == 0: continue

        match values[0]:
            case 'v': vertices.append(values[1:4])
            case 'vn': normals.append(values[1:4])
            case 'vt': texture_coords.append(values[1:3])
            case 'usemtl': material = values[1]
            case 'usemat': material = values[1]
            case 'f':
                face = []
                face_texture = []
                face_normals = []

                for v in values[1:]:

                    w = v.split('/')
                    face.append(int(w[0]))
                    face_normals.append(int(w[2]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)

                faces.append((face, face_texture, face_normals, material))

             
    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [ ]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura, img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.convert("RGBA").tobytes("raw", "RGBA", 0, -1)

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height,
                 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

In [ ]:
vertices_list = []
normals_list = []
textures_coord_list = []


class Coordinates(TypedDict):
    x = float
    y = float
    z = float


class Model:
    def __init__(self, obj: str, textures: list[int, str], angle: float, r: Coordinates, t: Coordinates, s: Coordinates):
        global vertices_list, textures_coord_list, normals_list

        self.model = load_model_from_file(obj)
        self.textures = textures
        self.angle = angle
        self.r = r
        self.t = t
        self.s = s

        self.vertices = []

        # Se o modelo só tem um vértice, nós utilizamos o primeiro e o último apenas
        if len(self.textures) == 1:
            self.vertices.append(len(vertices_list))

        # Guarda os vértices do modelo
        faces_visited = []
        for face in self.model['faces']:
            if len(self.textures) > 1 and face[3] not in faces_visited:
                self.vertices.append(len(vertices_list))
                faces_visited.append(face[3])

            # Conta a quantidade de vértices
            for vertice_id in face[0]:
                vertices_list.append(self.model['vertices'][vertice_id-1])

            for texture_id in face[1]:
                textures_coord_list.append(self.model['texture'][texture_id-1])

            for normal_id in face[2]:
                normals_list.append(self.model['normals'][normal_id-1])

        self.vertices.append(len(vertices_list))

        for t in textures:
            load_texture_from_file(t[0], t[1])

    def draw(self):
        mat_model = model(self.angle,
                          self.r['x'], self.r['y'], self.r['z'],
                          self.t['x'], self.t['y'], self.t['z'],
                          self.s['x'], self.s['y'], self.s['z'])

        loc_model = glGetUniformLocation(program, "model")
        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

        # Carrega as texturas para cada face
        for texture in self.textures:
            index = self.textures.index(texture)
            glBindTexture(GL_TEXTURE_2D, texture[0])
            glDrawArrays(GL_TRIANGLES,
                         self.vertices[index], self.vertices[index + 1] - self.vertices[index])

In [ ]:
# Guarda os modelos carregados
models = {
    'terreno': Model('terreno/terreno.obj', [(0, 'terreno/terreno.png')], 0.0,
                     Coordinates(x=0.0, y=0.0, z=1.0),
                     Coordinates(x=0.0, y=-1.01, z=0.0),
                     Coordinates(x=200.0, y=200.0, z=200.0)),

    'skybox': Model('skybox/skybox.obj', [(1, 'skybox/skybox.png')], 0.0,
                    Coordinates(x=0.0, y=1.0, z=0.0),
                    Coordinates(x=0.0, y=0.0, z=0.0),
                    Coordinates(x=3.0, y=3.0, z=3.0)),

    'caminho': Model('caminho/caminho.obj', [(2, 'caminho/caminho.jpg')], 0.0,
                     Coordinates(x=0.0, y=1.0, z=0.0),
                     Coordinates(x=-54.0, y=-0.95, z=0.0),
                     Coordinates(x=95.0, y=1.0, z=15.0)),

    'gato': Model('gato/gato.obj', [(3, 'gato/gato.jpg')], 90.0,
                  Coordinates(x=-1.0, y=0.0, z=0.0),
                  Coordinates(x=30.0, y=31.3, z=-3.7),
                  Coordinates(x=0.05, y=0.05, z=0.05)),

    'lobo': Model('lobo/lobo.obj', [(4, 'lobo/lobo.jpg')], -90.0,
                  Coordinates(x=0.0, y=1.0, z=0.0),
                  Coordinates(x=0.0, y=-1.0, z=0.0),
                  Coordinates(x=12.0, y=12.0, z=12.0)),

    'torre': Model('torre/torre.obj', [(5, 'torre/torre.jpg')], 0.0,
                   Coordinates(x=0.0, y=0.0, z=1.0),
                   Coordinates(x=30.0, y=-4.0, z=0.0),
                   Coordinates(x=5.0, y=5.0, z=5.0)),

    'arvore': Model('arvore/arvore.obj', [(6, 'arvore/casca.jpg'), (7, 'arvore/folhas.png')], 0.0,
                    Coordinates(x=0.0, y=0.0, z=1.0),
                    Coordinates(x=0.0, y=-1.0, z=0.0),
                    Coordinates(x=7.0, y=7.0, z=7.0)),

    'golem': Model('golem/golem.obj', [(8, 'golem/golem.tif')], 90.0,
                   Coordinates(x=0.0, y=1.0, z=0.0),
                   Coordinates(x=-110.0, y=-1.0, z=0.0),
                   Coordinates(x=2.5, y=2.5, z=2.5)),

    'gaveta': Model('gaveta/gaveta.obj', [(9, 'gaveta/gaveta.png')], 180.0,
                    Coordinates(x=0.0, y=1.0, z=0.0),
                    Coordinates(x=33.5, y=28.5, z=2.0),
                    Coordinates(x=1.0, y=1.0, z=1.0)),

    'caixa': Model('caixa/caixa.obj', [(10, 'caixa/caixa.jpg')], 0.0,
                   Coordinates(x=-1.0, y=0.0, z=0.0),
                   Coordinates(x=30.0, y=30.0, z=-4.0),
                   Coordinates(x=1.2, y=1.2, z=1.2)),
}

# Definição de Desenhos

In [ ]:
def desenha_modelos(skybox_inc: float,
                    lobo_x: float, lobo_z: float, lobo_scale: float, lobo_angle: float,
                    arvore_qtd: int, arvore_positions: list[float, float]):

    # Desenha de acordo com as especificações de cada modelo
    for name, model in models.items():
        match(name):
            case 'skybox':
                model.angle = skybox_inc
                model.draw()
                
            case 'lobo':
                model.angle = -90.0 + lobo_angle
                model.t = {'x': lobo_x, 'y': -1.0, 'z': lobo_z}
                model.s = {'x': 12 + lobo_scale, 'y': 12 + lobo_scale, 'z': 12 + lobo_scale}
                model.draw()
            
            case 'arvore':
                for i in range(arvore_qtd):
                    model.t = {'x': arvore_positions[i][0], 'y': -1.0, 'z': arvore_positions[i][1]}
                    model.draw()
            
            case _:
                model.draw()

# Rodando

### Uso da GPU

In [ ]:
buffer = glGenBuffers(3)

shader_binds = {
    'position': 0,
    'texture_coord': 1,
    'normals': 2
}

In [ ]:
def bind_buffer(values_list, name):
    size = len(values_list[0])
    
    values = np.zeros(len(values_list), [("position", np.float32, size)])
    values['position'] = values_list
    
    stride = values.strides[0]
    offset = ctypes.c_void_p(0)

    glBindBuffer(GL_ARRAY_BUFFER, buffer[shader_binds[name]])
    glBufferData(GL_ARRAY_BUFFER, values.nbytes, values, GL_STATIC_DRAW)

    glBindAttribLocation(program, shader_binds[name], name)
    glEnableVertexAttribArray(shader_binds[name])
    glVertexAttribPointer(shader_binds[name], size, GL_FLOAT, False, stride, offset)

In [ ]:
bind_buffer(vertices_list, 'position')
bind_buffer(textures_coord_list, 'texture_coord')
bind_buffer(normals_list, 'normals')

### Eventos

In [ ]:
camera_pos = glm.vec3(25.0,  33.0,  0.0)
camera_front = glm.vec3(0.0,  0.0, -1.0)
camera_up = glm.vec3(0.0,  1.0,  0.0)

trap_mouse = True
polygonal_mode = False

inc_fov = 0
inc_near = 0
inc_far = 0
inc_view_up = 0

x_lobo = 0
z_lobo = 0
scale_lobo = 0
rotate_lobo = 0


def check_validity(position):
    camera_abs = math.sqrt((position.x ** 2) +
                           (position.y ** 2) + (position.z ** 2))
    if (camera_abs < 138.0) and position.y >= 0:
        return True
    else:
        return False


def key_event(window, key, scancode, action, mods):
    global camera_pos, camera_front, camera_up, polygonal_mode, trap_mouse, inc_fov, inc_near, inc_far, camera_up, inc_view_up
    global x_lobo, z_lobo, scale_lobo, rotate_lobo

    camera_speed = 0.75

    if (action == 1 or action == 2):
        match key:
            case 87:  # W
                if check_validity(camera_pos + (camera_speed * camera_front)):
                    camera_pos += camera_speed * camera_front

            case 83:  # S
                if check_validity(camera_pos - (camera_speed * camera_front)):
                    camera_pos -= camera_speed * camera_front

            case 65:  # A
                if check_validity(camera_pos - glm.normalize(glm.cross(camera_front, camera_up)) * camera_speed):
                    camera_pos -= glm.normalize(glm.cross(camera_front,
                                                          camera_up)) * camera_speed

            case 68:  # D
                if check_validity(camera_pos + glm.normalize(glm.cross(camera_front, camera_up)) * camera_speed):
                    camera_pos += glm.normalize(glm.cross(camera_front,
                                                          camera_up)) * camera_speed

            case 80:  # P
                if action != 2:
                    polygonal_mode = not polygonal_mode

            case 77:  # M
                if action != 2:
                    trap_mouse = not trap_mouse

            case 265:  # ArrowUp
                x_lobo -= 0.5

            case 264:  # ArrowDown
                x_lobo += 0.5

            case 263:  # ArrowLeft
                z_lobo += 0.25

            case 262:  # ArrowRight
                z_lobo -= 0.25

            case 90:  # Z
                scale_lobo += 0.5

            case 88:  # X
                scale_lobo -= 0.5

            case 67:  # C
                rotate_lobo -= 3

            case 86:  # V
                rotate_lobo += 3


first_mouse = True
yaw = -33.7
pitch = 5.9
last_x = largura / 2
last_y = altura / 2


def mouse_event(window, xpos, ypos):
    global first_mouse, camera_front, yaw, pitch, last_x, last_y

    if first_mouse:
        last_x = xpos
        last_y = ypos
        first_mouse = False

    xoffset = xpos - last_x
    yoffset = last_y - ypos
    last_x = xpos
    last_y = ypos

    sensitivity = 0.3
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    if pitch >= 89.9:
        pitch = 89.9
    if pitch <= -89.9:
        pitch = -89.9

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    camera_front = glm.normalize(front)


glfw.set_key_callback(window, key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Execução Final

In [ ]:
glfw.show_window(window)
glfw.set_cursor_pos(window, last_x, last_y)

In [ ]:
glEnable(GL_DEPTH_TEST)
glEnable(GL_ALPHA_TEST)
glAlphaFunc(GL_GREATER, 0.1)

rotacao_inc = 0
while not glfw.window_should_close(window):
    glfw.poll_events()

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    glClearColor(1.0, 1.0, 1.0, 1.0)

    if polygonal_mode == True:
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)

    if trap_mouse == True:
        glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)
    else:
        glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_NORMAL)

    rotacao_inc += 0.1

    arvores = []
    value = 5
    for i in range(7):
        arvores.append((value - 20*i, 20))
        arvores.append((value - 20*i, -20))
        arvores.append((value - 20*i, 40))
        arvores.append((value - 20*i, -40))
        arvores.append((value - 20*i, 60))
        arvores.append((value - 20*i, -60))

    desenha_modelos(rotacao_inc, x_lobo, z_lobo, scale_lobo,
                    rotate_lobo, len(arvores), arvores)

    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")

    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)

    glfw.swap_buffers(window)

glfw.terminate()